# Use SX5E and bloomberg download data to get me started

In [46]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [47]:
#initial cleaning function
def clean_sheet(dataframe):
    adj=dataframe.dropna(axis=1,how='all')
    adj=dataframe.set_index(adj.columns[0]) #use stock tickers as the index
    return adj

In [48]:
#pull the data from excel, form a dictionary of tab names and dataframes
tabs_to_grab=['Quarter','Date','Reported EPS','Comp EPS','Consensus EPS','50d MA','3d return','3d rel return','Volume','Momentum',
             'Revision','Market cap','PE','Property','Price']

data_dict={}

for s in tabs_to_grab:
    data_dict[s]=clean_sheet(pd.read_excel(r'C:\Users\Eric.Li\Documents\Post result data\2016-2018.xlsx',sheet_name=s))

In [49]:
# grab the price time series and transform it into the same datetime format; also generate abs_return, rel_return dataframe
price=data_dict["Price"].dropna(axis=1)
new_column=[i.strftime("%d/%b/%Y") for i in price.columns.tolist() if type(i)!=str]
price.columns=new_column


## Signal 2 - based on last quarter's standing

In [127]:
## grab the date 
date=data_dict["Date"].dropna(axis=1,how='all')
adj_date=date.fillna(pd.Timestamp('1900-1-1'))


In [183]:
## form the signal 2: separate date range into a unit of 3 months, evaluate the the top and bottom quartile values, use the
# dataframe for the next quarter to generate signals, and put them into long and short buckets
start_date=pd.Timestamp(2015,12,31)
end_date=pd.Timestamp(2018,3,31)
per='Q'


def signal2_form(date_df,return_df,start_date,end_date,top_quartile,bottom_quartile):
    dt_rng=pd.date_range(start_date,end_date,freq=per)
    long_list=[]
    short_list=[]
    for s in range(len(dt_rng)-2):
        rank_list=[]
        rank=return_df[(date_df>=dt_rng[s])&(date_df<=dt_rng[s+1])]
        signal=date_df[(date_df>=dt_rng[s+1])&(date_df<=dt_rng[s+2])]
        signal_value=return_df[(date_df>=dt_rng[s+1])&(date_df<=dt_rng[s+2])]
        for index,row in rank.iterrows():
            adj_row=row.dropna()
            for val in adj_row.values:
                rank_list.append(val)
        rank_series=pd.Series(rank_list)
        top=signal[signal_value>=np.percentile(rank_series,top_quartile)].dropna(axis=1,how='all').dropna(axis=0,how='all')
        bottom=signal[signal_value<=np.percentile(rank_series,bottom_quartile)].dropna(axis=1,how='all').dropna(axis=0,how='all')
        long_list.append(top)
        short_list.append(bottom)
    long_signal=pd.concat(long_list,axis=1)
    short_signal=pd.concat(short_list,axis=1)
    return long_signal, short_signal
                
        
    
signal2_form(adj_date,data_dict["3d rel return"],start_date,end_date,80,20)   


(         Unnamed: 7 Unnamed: 8 Unnamed: 4 Unnamed: 6 Unnamed: 7 Unnamed: 6  \
 ABI BB          NaT        NaT        NaT        NaT 2016-07-29        NaT   
 AD NA           NaT 2016-06-01        NaT        NaT 2016-08-25        NaT   
 ADS GY          NaT        NaT        NaT        NaT        NaT        NaT   
 AIR FP          NaT        NaT        NaT        NaT        NaT        NaT   
 ALV GY          NaT        NaT        NaT        NaT 2016-08-05        NaT   
 ASML NA         NaT        NaT        NaT        NaT        NaT        NaT   
 BAS GY          NaT        NaT        NaT        NaT        NaT        NaT   
 BAYN GY         NaT        NaT        NaT        NaT 2016-07-27        NaT   
 BN FP           NaT        NaT 2016-07-28        NaT        NaT        NaT   
 CRH ID          NaT        NaT 2016-08-25        NaT        NaT        NaT   
 DG FP           NaT        NaT 2016-07-29        NaT        NaT        NaT   
 DPW GY          NaT 2016-05-11        NaT        Na

In [132]:
dt_rng

DatetimeIndex(['2015-12-31', '2016-03-31', '2016-06-30', '2016-09-30',
               '2016-12-31', '2017-03-31', '2017-06-30', '2017-09-30',
               '2017-12-31', '2018-03-31'],
              dtype='datetime64[ns]', freq='Q-DEC')

In [128]:
start_date=pd.Timestamp(2015,12,31)
end_date=pd.Timestamp(2018,3,31)
per='Q'
date_df=adj_date
return_df=data_dict["3d rel return"]

dt_rng=pd.date_range(start_date,end_date,freq=per)
signal_dict={}

dt_rng

DatetimeIndex(['2015-12-31', '2016-03-31', '2016-06-30', '2016-09-30',
               '2016-12-31', '2017-03-31', '2017-06-30', '2017-09-30',
               '2017-12-31', '2018-03-31'],
              dtype='datetime64[ns]', freq='Q-DEC')

In [167]:
top_quartile=80
bottom_quartile=20

In [168]:
dt_rng=pd.date_range(start_date,end_date,freq=per)
long_list=[]
short_list=[]
for s in range(len(dt_rng)-2):
    rank_list=[]
    rank=return_df[(date_df>=dt_rng[s])&(date_df<=dt_rng[s+1])]
    signal=date_df[(date_df>=dt_rng[s+1])&(date_df<=dt_rng[s+2])]
    for index,row in rank.iterrows():
        adj_row=row.dropna()
        for val in adj_row.values:
            rank_list.append(val)
    rank_series=pd.Series(rank_list)
    top=signal[signal>=np.percentile(rank_series,top_quartile)].dropna(axis=1,how='all').dropna(axis=0,how='all')
    bottom=signal[signal<=np.percentile(rank_series,bottom_quartile)].dropna(axis=1,how='all').dropna(axis=0,how='all')
    long_list.append(top)
    short_list.append(bottom)
long_signal=pd.concat(long_list,axis=1)
short_signal=pd.concat(short_list,axis=1)

In [181]:
top

,2018-03-12 00:00:00
Unnamed: 0,
ABI BB,2018-03-01
AD NA,2018-02-28
AI FP,2018-02-15
AIR FP,2018-02-15
ALV GY,2018-02-16
ASML NA,2018-01-17
BAS GY,2018-02-27
BAYN GY,2018-02-28
BBVA SQ,2018-02-01


In [149]:
dt_rng[s]

ValueError: 

In [138]:
for index,row in rank.iterrows():
    adj_row=row.dropna()
    for s in adj_row.values:
        rank_list.append(s)

In [133]:
dt_rng[1]

Timestamp('2016-03-31 00:00:00', freq='Q-DEC')

In [139]:
rank_list

[1.2813082315486426,
 4.572702209605447,
 0.8750412188635015,
 -4.7867815316928315,
 -0.9070412416909215,
 -2.5681052673479488,
 -2.506432829267509,
 -5.025287312197128,
 -3.7763628061676187,
 -2.184375602364504,
 -0.13773521523920582,
 -2.0907849425614073,
 -0.5742527460149079,
 3.712569084253186,
 -0.041623463478806834,
 0.6246167797856443,
 1.119272282092696,
 -2.389681055751796,
 -0.49320938114280316,
 1.5941954335986437,
 0.29505959622389116,
 0.5669968959956173,
 3.055685541572396,
 0.8524199847235193,
 1.2550727279995282,
 3.6617009100282036,
 -2.0151509488739126,
 -6.676433185038674,
 -0.16259533663604236,
 -2.545918501760307,
 -2.440774470185061,
 -0.25907748994380103,
 0.4491855515895614,
 2.7934824355979964,
 -2.2609297070779966,
 0.7525668689752891,
 -0.6480847561201544,
 -0.36030821806654256,
 1.2813082315486426,
 4.572702209605447,
 0.8750412188635015,
 -4.7867815316928315,
 -0.9070412416909215,
 -2.5681052673479488,
 -2.506432829267509,
 -5.025287312197128,
 -3.776362806